# SynFerm data preparation
#### Targets:
- Import experiment, representation, and target data from db
- Export to CSV

In [4]:
import os
import sys

sys.path.insert(0, os.path.abspath('../..'))
from copy import deepcopy

from IPython.display import SVG, display
import pandas as pd

from src.util.db_utils import SynFermDatabaseConnection


In [5]:
con = SynFermDatabaseConnection()  # we will use this for various simple queries

In [7]:
res = con.con.execute('SELECT e.id, r.I_long, r.M_long, r.T_long, r.product_A_smiles, r.I_smiles, r.M_smiles, r.T_smiles, r.reaction_smiles, r.reaction_smiles_atom_mapped, l.binary_A, l.binary_B, l.binary_C, l.binary_D, l.binary_E, l.binary_F, l.binary_G, l.binary_H, l.scaled_A, l.scaled_B, l.scaled_C, l.scaled_D, l.scaled_E, l.scaled_F, l.scaled_G, l.scaled_H, l."major_A-C" FROM experiments e LEFT JOIN representations r on e.id = r.experiment_id LEFT JOIN labels l on e.id = l.experiment_id;').fetchall()

columns = ["experiment_id", "I_long", "M_long", "T_long", "product_A_smiles", "I_smiles", "M_smiles", "T_smiles", "reaction_smiles", "reaction_smiles_atom_mapped", "binary_A", "binary_B", "binary_C", "binary_D", "binary_E", "binary_F", "binary_G", "binary_H", "scaled_A", "scaled_B", "scaled_C", "scaled_D", "scaled_E", "scaled_F", "scaled_G", "scaled_H", "major_A-C"]
df_full = pd.DataFrame(res, columns=columns)
print(f'Number of reactions (in total): {len(df_full)}')

Number of reactions (in total): 58664


In [8]:
df_valid = df_full.loc[(~df_full["valid"].str.contains("ERROR", na=False)) & (df_full["exp_nr"].between(4, 29))]  # kick out invalid experiments
print(f'Number of valid (non-test) reactions: {len(df_valid)}')

KeyError: 'valid'

In [23]:
# doublecheck we don't have missing values
df_valid['product_A_lcms_ratio'].isna().sum()

0

Now we have a cleaned dataset. Export to CSV.

In [ ]:
df_valid.to_csv(f'synferm_dataset_2023-07-20_{len(df_valid)}records.csv', index=False)